In [1]:
validate_or_submit = 'submit'
verbose = True
SEED = 42

import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import warnings
import json
import os
import gc
from collections import defaultdict
import polars as pl
from scipy import signal, stats
from scipy.ndimage import gaussian_filter1d
import psutil
import pickle

from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')
np.random.seed(SEED)

from catboost import CatBoostClassifier

try:
    import cupy as cp
    CUPY_AVAILABLE = True
    print("CuPy available - GPU acceleration enabled")
except:
    CUPY_AVAILABLE = False
    print("CuPy not available - using CPU fallback")

print(f"Available RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"Current usage: {psutil.virtual_memory().percent}%")

if CUPY_AVAILABLE:
    print(f"GPU Memory: {cp.cuda.Device(0).mem_info[1] / (1024**3):.1f} GB")

CuPy available - GPU acceleration enabled
Available RAM: 31.4 GB
Current usage: 4.0%
GPU Memory: 15.9 GB


In [2]:
class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    def __init__(self, estimator, n_samples, random_state=42):
        self.estimator = estimator
        self.n_samples = n_samples
        self.random_state = random_state

    def fit(self, X, y):
        if len(X) <= self.n_samples:
            self.estimator.fit(X, y)
        else:
            from sklearn.model_selection import StratifiedShuffleSplit
            sss = StratifiedShuffleSplit(n_splits=1, train_size=min(self.n_samples, len(X)), random_state=self.random_state)
            try:
                for train_idx, _ in sss.split(X, y):
                    X_sample = X.iloc[train_idx] if hasattr(X, 'iloc') else X[train_idx]
                    y_sample = y[train_idx]
                    self.estimator.fit(X_sample, y_sample)
                    del X_sample, y_sample
            except:
                np.random.seed(self.random_state)
                downsample = max(len(X) // self.n_samples, 1)
                X_ds = X[::downsample]
                y_ds = y[::downsample]
                self.estimator.fit(X_ds, y_ds)
                del X_ds, y_ds

        self.classes_ = self.estimator.classes_
        gc.collect()
        if CUPY_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()
        return self

    def predict_proba(self, X):
        if len(self.classes_) == 1:
            return np.full((len(X), 1), 1.0)
        proba = self.estimator.predict_proba(X)
        return proba

    def predict(self, X):
        pred = self.estimator.predict(X)
        return pred

In [3]:
train = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~ lab_id.str.startswith('MABe22_')")

test = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

drop_body_parts = ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright',
                   'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright',
                   'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

arena_metadata = {}
for _, row in pd.concat([train, test]).iterrows():
    arena_metadata[row['video_id']] = {
        'pix_per_cm': row['pix_per_cm_approx'],
        'fps': row.get('fps', 30),
        'lab_id': row['lab_id']
    }

gc.collect()
print(f"Memory after loading: {psutil.virtual_memory().percent}%")

Memory after loading: 4.4%


In [4]:
def normalize_coordinates(x, y, pix_per_cm, arena_width_cm=120, arena_height_cm=120):
    x_norm = np.clip(x / pix_per_cm, 0, arena_width_cm)
    y_norm = np.clip(y / pix_per_cm, 0, arena_height_cm)
    return x_norm, y_norm

def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"
    for _, row in dataset.iterrows():

        lab_id = row.lab_id
        if lab_id.startswith('MABe22'):
            continue
        video_id = row.video_id

        if type(row.behaviors_labeled) != str:
            if verbose:
                print('No labeled behaviors:', lab_id, video_id)
            continue

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"

        try:
            vid = pd.read_parquet(path, columns=['video_frame', 'mouse_id', 'bodypart', 'x', 'y'])
        except:
            continue

        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")

        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])

        if pvid.isna().any().any():
            if verbose and traintest == 'test':
                print('video with missing values', video_id, traintest, len(vid), 'frames')

        del vid
        gc.collect()

        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        pvid = pvid.astype(np.float32)

        for mouse_id in np.unique(pvid.columns.get_level_values('mouse_id')):
            for bodypart_name in pvid[mouse_id].columns.get_level_values(0).unique():
                if 'x' in pvid[mouse_id][bodypart_name].columns:
                    x_vals = pvid[(mouse_id, bodypart_name, 'x')]
                    y_vals = pvid[(mouse_id, bodypart_name, 'y')]
                    x_norm, y_norm = normalize_coordinates(x_vals, y_vals, row.pix_per_cm_approx)
                    pvid[(mouse_id, bodypart_name, 'x')] = x_norm
                    pvid[(mouse_id, bodypart_name, 'y')] = y_norm

        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])

        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                del pvid
                gc.collect()
                continue

        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'")
            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                    single_mouse = pvid.loc[:, mouse_id].copy()
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index,
                        'fps': row.get('fps', 30)
                    })
                    if traintest == 'train':
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index, dtype=np.float32)
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose:
                            print('- test single', video_id, mouse_id)
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                except KeyError:
                    pass

        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    if len(vid_agent_actions) == 0:
                        continue

                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B']).copy()
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index,
                        'fps': row.get('fps', 30)
                    })
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index, dtype=np.float32)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose:
                            print('- test pair', video_id, agent, target)
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions

        del pvid
        gc.collect()

In [5]:
def safe_rolling(series, window, func, min_periods=None):
    if min_periods is None:
        min_periods = max(1, window // 4)
    return series.rolling(window, min_periods=min_periods, center=True).apply(func, raw=True)

def add_spectral_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)

    for window in [60, 120]:
        if len(speed) >= window:
            speed_chunk = speed.rolling(window, min_periods=window//2).apply(
                lambda x: np.sum(np.abs(np.fft.rfft(x - x.mean())[:5])), raw=True
            )
            X[f'fft_pwr{window}'] = speed_chunk

    del speed
    gc.collect()
    return X

def add_curvature_features(X, center_x, center_y):
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)

    X['curv_m30'] = curvature.rolling(30, min_periods=5).mean()
    X['curv_m60'] = curvature.rolling(60, min_periods=10).mean()
    X['curv_s30'] = curvature.rolling(30, min_periods=5).std()
    X['curv_max60'] = curvature.rolling(60, min_periods=10).max()

    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    X['turn_r30'] = angle_change.rolling(30, min_periods=5).sum()
    X['turn_r60'] = angle_change.rolling(60, min_periods=10).sum()
    X['turn_m30'] = angle_change.rolling(30, min_periods=5).mean()
    X['turn_std30'] = angle_change.rolling(30, min_periods=5).std()

    del vel_x, vel_y, acc_x, acc_y, cross_prod, vel_mag, curvature, angle, angle_change
    gc.collect()

    return X

def add_multiscale_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)

    scales = [5, 10, 20, 40, 80, 160]
    for scale in scales:
        if len(speed) >= scale:
            min_p = max(1, scale//4)
            X[f'sp_m{scale}'] = speed.rolling(scale, min_periods=min_p).mean()
            X[f'sp_s{scale}'] = speed.rolling(scale, min_periods=min_p).std()
            X[f'sp_mx{scale}'] = speed.rolling(scale, min_periods=min_p).max()
            X[f'sp_mn{scale}'] = speed.rolling(scale, min_periods=min_p).min()

    if len(scales) >= 2:
        for i in range(len(scales)-1):
            if f'sp_m{scales[i]}' in X.columns and f'sp_m{scales[i+1]}' in X.columns:
                X[f'sp_rt{scales[i]}_{scales[i+1]}'] = X[f'sp_m{scales[i]}'] / (X[f'sp_m{scales[i+1]}'] + 1e-6)

    del speed
    gc.collect()

    return X

def add_state_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)
    speed_ma = speed.rolling(15, min_periods=5).mean()

    try:
        speed_states = pd.cut(speed_ma, bins=[-np.inf, 0.5, 2.0, 5.0, np.inf], labels=[0, 1, 2, 3]).astype(float)

        for window in [30, 60, 120, 240]:
            if len(speed_states) >= window:
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(window, min_periods=10).sum()
                X[f'state_m{window}'] = speed_states.rolling(window, min_periods=10).mean()
                X[f'state_s{window}'] = speed_states.rolling(window, min_periods=10).std()

        del speed_states, state_changes
    except:
        pass

    del speed, speed_ma
    gc.collect()

    return X

def add_longrange_features(X, center_x, center_y):
    for window in [60, 120, 240]:
        if len(center_x) >= window:
            X[f'x_ml{window}'] = center_x.rolling(window, min_periods=20).mean()
            X[f'y_ml{window}'] = center_y.rolling(window, min_periods=20).mean()
            X[f'x_sl{window}'] = center_x.rolling(window, min_periods=20).std()
            X[f'y_sl{window}'] = center_y.rolling(window, min_periods=20).std()

    for span in [30, 60, 120]:
        X[f'x_e{span}'] = center_x.ewm(span=span, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=span, min_periods=1).mean()

    dist_from_center = np.sqrt((center_x - center_x.mean())**2 + (center_y - center_y.mean())**2)
    for window in [30, 60, 120]:
        X[f'cen_d{window}'] = dist_from_center.rolling(window, min_periods=5).mean()
        X[f'cen_s{window}'] = dist_from_center.rolling(window, min_periods=5).std()

    del dist_from_center
    gc.collect()

    return X

def add_interaction_features(X, mouse_pair, avail_A, avail_B):
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()

    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    for window in [15, 30, 60, 120]:
        X[f'A_ld{window}'] = A_lead.rolling(window, min_periods=5).mean()
        X[f'B_ld{window}'] = B_lead.rolling(window, min_periods=5).mean()
        X[f'A_ld_s{window}'] = A_lead.rolling(window, min_periods=5).std()
        X[f'B_ld_s{window}'] = B_lead.rolling(window, min_periods=5).std()

    approach = -rel_dist.diff()
    chase = approach * B_lead
    for window in [15, 30, 60, 120]:
        X[f'chase_{window}'] = chase.rolling(window, min_periods=5).mean()
        X[f'appr_{window}'] = approach.rolling(window, min_periods=5).mean()
        X[f'chase_s{window}'] = chase.rolling(window, min_periods=5).std()

    rel_angle = np.arctan2(rel_y, rel_x)
    rel_angle_change = np.abs(rel_angle.diff())
    for window in [30, 60, 120]:
        X[f'rel_ang{window}'] = rel_angle_change.rolling(window, min_periods=5).sum()
        X[f'rel_ang_m{window}'] = rel_angle_change.rolling(window, min_periods=5).mean()

    del rel_x, rel_y, rel_dist, A_vx, A_vy, B_vx, B_vy, A_lead, B_lead, approach, chase, rel_angle, rel_angle_change
    gc.collect()

    return X

def add_spatial_features(X, center_x, center_y):
    grid_x = (center_x / 10).fillna(0).astype(int)
    grid_y = (center_y / 10).fillna(0).astype(int)

    for window in [60, 120, 240]:
        if len(grid_x) >= window:
            grid_changes = ((grid_x != grid_x.shift(1)) | (grid_y != grid_y.shift(1))).astype(float)
            X[f'grid_ch{window}'] = grid_changes.rolling(window, min_periods=10).sum()

    X['x_quad'] = (center_x > center_x.median()).astype(float)
    X['y_quad'] = (center_y > center_y.median()).astype(float)

    wall_dist = np.minimum(
        np.minimum(center_x, 120 - center_x),
        np.minimum(center_y, 120 - center_y)
    )
    X['wall_d'] = wall_dist
    X['wall_d30'] = wall_dist.rolling(30, min_periods=5).mean()

    del grid_x, grid_y, wall_dist
    gc.collect()

    return X

def add_advanced_temporal_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)
    
    for window in [10, 20, 40]:
        X[f'speed_accel_{window}'] = speed.diff().rolling(window, min_periods=5).mean()
        X[f'speed_jerk_{window}'] = speed.diff().diff().rolling(window, min_periods=5).mean()
    
    for window in [30, 60]:
        X[f'speed_skew_{window}'] = speed.rolling(window, min_periods=10).skew()
        X[f'speed_kurt_{window}'] = speed.rolling(window, min_periods=10).kurt()
    
    del speed
    gc.collect()
    return X

def add_posture_features(X, single_mouse):
    if all(p in single_mouse.columns for p in ['nose', 'body_center', 'tail_base', 'ear_left', 'ear_right']):
        head_width = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                            (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        body_length = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                             (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        
        X['aspect_ratio'] = body_length / (head_width + 1e-6)
        
        for window in [15, 30, 60]:
            X[f'aspect_m{window}'] = X['aspect_ratio'].rolling(window, min_periods=5).mean()
            X[f'aspect_s{window}'] = X['aspect_ratio'].rolling(window, min_periods=5).std()
        
        del head_width, body_length
        gc.collect()
    
    return X

def add_behavioral_rhythm_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)
    
    for window in [90, 180]:
        if len(speed) >= window:
            fft_vals = speed.rolling(window, min_periods=window//2).apply(
                lambda x: np.abs(np.fft.rfft(x - x.mean())[1:6]).max() if len(x) >= 10 else 0, raw=True
            )
            X[f'rhythm_peak_{window}'] = fft_vals
    
    for window in [60, 120]:
        active = (speed > speed.quantile(0.6)).astype(float)
        X[f'active_ratio_{window}'] = active.rolling(window, min_periods=10).mean()
        del active
        gc.collect()
    
    del speed
    gc.collect()
    return X

def add_pair_synchrony_features(X, mouse_pair, avail_A, avail_B):
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X
    
    A_speed = np.sqrt(mouse_pair['A']['body_center']['x'].diff()**2 + 
                     mouse_pair['A']['body_center']['y'].diff()**2)
    B_speed = np.sqrt(mouse_pair['B']['body_center']['x'].diff()**2 + 
                     mouse_pair['B']['body_center']['y'].diff()**2)
    
    for window in [30, 60, 120]:
        A_norm = (A_speed - A_speed.rolling(window, min_periods=10).mean()) / (A_speed.rolling(window, min_periods=10).std() + 1e-6)
        B_norm = (B_speed - B_speed.rolling(window, min_periods=10).mean()) / (B_speed.rolling(window, min_periods=10).std() + 1e-6)
        X[f'sync_{window}'] = (A_norm * B_norm).rolling(window, min_periods=10).mean()
        del A_norm, B_norm
        gc.collect()
    
    for window in [30, 60]:
        A_active = (A_speed > A_speed.quantile(0.6)).astype(float)
        B_active = (B_speed > B_speed.quantile(0.6)).astype(float)
        X[f'co_active_{window}'] = (A_active * B_active).rolling(window, min_periods=5).mean()
        del A_active, B_active
        gc.collect()
    
    speed_diff = np.abs(A_speed - B_speed)
    for window in [30, 60]:
        X[f'speed_diff_m{window}'] = speed_diff.rolling(window, min_periods=5).mean()
        X[f'speed_diff_s{window}'] = speed_diff.rolling(window, min_periods=5).std()
    
    del A_speed, B_speed, speed_diff
    gc.collect()
    return X

def add_spatial_context_features(X, center_x, center_y):
    for window in [60, 120, 240]:
        if len(center_x) >= window:
            X[f'area_covered_{window}'] = (center_x.rolling(window, min_periods=10).max() - center_x.rolling(window, min_periods=10).min()) * \
                                          (center_y.rolling(window, min_periods=10).max() - center_y.rolling(window, min_periods=10).min())
    
    arena_center_x = 60.0
    arena_center_y = 60.0
    dist_to_arena_center = np.sqrt((center_x - arena_center_x)**2 + (center_y - arena_center_y)**2)
    
    for window in [30, 60, 120]:
        X[f'center_pref_{window}'] = (dist_to_arena_center < 30).astype(float).rolling(window, min_periods=5).mean()
    
    del dist_to_arena_center
    gc.collect()
    return X

In [6]:
def add_momentum_features(X, center_x, center_y):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2)
    accel = speed.diff()
    
    for window in [10, 20, 40, 80]:
        momentum = speed.rolling(window, min_periods=5).mean() * accel.rolling(window, min_periods=5).mean()
        X[f'momentum_{window}'] = momentum
        
        speed_ewm = speed.ewm(span=window, min_periods=5).mean()
        X[f'speed_momentum_{window}'] = speed_ewm * accel
        
        del momentum, speed_ewm
        gc.collect()
    
    del speed, accel
    gc.collect()
    return X

def add_directional_features(X, center_x, center_y):
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    
    direction = np.arctan2(vel_y, vel_x)
    direction_change = direction.diff()
    
    for window in [15, 30, 60, 120]:
        X[f'dir_std_{window}'] = direction.rolling(window, min_periods=5).std()
        X[f'dir_range_{window}'] = direction.rolling(window, min_periods=5).max() - direction.rolling(window, min_periods=5).min()
        X[f'dir_change_sum_{window}'] = np.abs(direction_change).rolling(window, min_periods=5).sum()
        
    persistence = (direction.diff().abs() < 0.1).astype(float)
    for window in [30, 60]:
        X[f'dir_persist_{window}'] = persistence.rolling(window, min_periods=5).mean()
    
    del vel_x, vel_y, direction, direction_change, persistence
    gc.collect()
    return X

def add_acceleration_patterns(X, center_x, center_y):
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()
    
    acc_mag = np.sqrt(acc_x**2 + acc_y**2)
    
    for window in [10, 20, 40]:
        X[f'acc_mean_{window}'] = acc_mag.rolling(window, min_periods=5).mean()
        X[f'acc_std_{window}'] = acc_mag.rolling(window, min_periods=5).std()
        X[f'acc_max_{window}'] = acc_mag.rolling(window, min_periods=5).max()
        
        burst = (acc_mag > acc_mag.quantile(0.75)).astype(float)
        X[f'acc_burst_{window}'] = burst.rolling(window, min_periods=5).sum()
        del burst
        gc.collect()
    
    del vel_x, vel_y, acc_x, acc_y, acc_mag
    gc.collect()
    return X

def add_relative_motion_features(X, mouse_pair, avail_A, avail_B):
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X
    
    A_vel_x = mouse_pair['A']['body_center']['x'].diff()
    A_vel_y = mouse_pair['A']['body_center']['y'].diff()
    B_vel_x = mouse_pair['B']['body_center']['x'].diff()
    B_vel_y = mouse_pair['B']['body_center']['y'].diff()
    
    A_speed = np.sqrt(A_vel_x**2 + A_vel_y**2)
    B_speed = np.sqrt(B_vel_x**2 + B_vel_y**2)
    
    rel_vel_x = A_vel_x - B_vel_x
    rel_vel_y = A_vel_y - B_vel_y
    rel_speed = np.sqrt(rel_vel_x**2 + rel_vel_y**2)
    
    for window in [15, 30, 60]:
        X[f'rel_speed_m_{window}'] = rel_speed.rolling(window, min_periods=5).mean()
        X[f'rel_speed_s_{window}'] = rel_speed.rolling(window, min_periods=5).std()
        
        speed_correlation = (A_speed * B_speed).rolling(window, min_periods=5).mean()
        X[f'speed_corr_{window}'] = speed_correlation / (A_speed.rolling(window, min_periods=5).std() * B_speed.rolling(window, min_periods=5).std() + 1e-6)
        
        del speed_correlation
        gc.collect()
    
    del A_vel_x, A_vel_y, B_vel_x, B_vel_y, A_speed, B_speed, rel_vel_x, rel_vel_y, rel_speed
    gc.collect()
    return X

In [7]:
def transform_single(single_mouse, body_parts_tracked):
    available_body_parts = single_mouse.columns.get_level_values(0)

    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    }, dtype=np.float32)
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        for shift in [5, 10, 20]:
            shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(shift)
            speeds = pd.DataFrame({
                f'sp_lf{shift}': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
                f'sp_rt{shift}': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
                f'sp_tb{shift}': np.square(single_mouse['tail_base'] - shifted['tail_base']).sum(axis=1, skipna=False),
            }, dtype=np.float32)
            X = pd.concat([X, speeds], axis=1)
            del shifted, speeds
            gc.collect()

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)
        X['elong_inv'] = X['ear_left+ear_right'] / (X['nose+tail_base'] + 1e-6)

    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

        for window in [15, 30, 60, 120]:
            X[f'body_ang_m{window}'] = X['body_ang'].rolling(window, min_periods=5).mean()
            X[f'body_ang_s{window}'] = X['body_ang'].rolling(window, min_periods=5).std()

        del v1, v2
        gc.collect()

    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 10, 15, 30, 60, 120]:
            X[f'cx_m{w}'] = cx.rolling(w, min_periods=1, center=True).mean()
            X[f'cy_m{w}'] = cy.rolling(w, min_periods=1, center=True).mean()
            X[f'cx_s{w}'] = cx.rolling(w, min_periods=1, center=True).std()
            X[f'cy_s{w}'] = cy.rolling(w, min_periods=1, center=True).std()
            X[f'x_rng{w}'] = cx.rolling(w, min_periods=1, center=True).max() - cx.rolling(w, min_periods=1, center=True).min()
            X[f'y_rng{w}'] = cy.rolling(w, min_periods=1, center=True).max() - cy.rolling(w, min_periods=1, center=True).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(w, min_periods=1).sum()**2 + cy.diff().rolling(w, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(w, min_periods=1).var() + cy.diff().rolling(w, min_periods=1).var())

        X = add_curvature_features(X, cx, cy)
        X = add_multiscale_features(X, cx, cy)
        X = add_state_features(X, cx, cy)
        X = add_longrange_features(X, cx, cy)
        X = add_spatial_features(X, cx, cy)
        X = add_spectral_features(X, cx, cy)
        X = add_advanced_temporal_features(X, cx, cy)
        X = add_posture_features(X, single_mouse)
        X = add_behavioral_rhythm_features(X, cx, cy)
        X = add_spatial_context_features(X, cx, cy)
        X = add_momentum_features(X, cx, cy)
        X = add_directional_features(X, cx, cy)
        X = add_acceleration_patterns(X, cx, cy)

    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                         (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)

        for lag in [5, 10, 20, 40, 80]:
            X[f'nt_lg{lag}'] = nt_dist.shift(lag)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(lag)

        for window in [30, 60, 120]:
            X[f'nt_m{window}'] = nt_dist.rolling(window, min_periods=5).mean()
            X[f'nt_s{window}'] = nt_dist.rolling(window, min_periods=5).std()
            X[f'nt_mx{window}'] = nt_dist.rolling(window, min_periods=5).max()

        del nt_dist
        gc.collect()

    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                       (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)

        for off in [-40, -20, -10, 10, 20, 40]:
            X[f'ear_o{off}'] = ear_d.shift(-off)

        for window in [30, 60, 120]:
            X[f'ear_m{window}'] = ear_d.rolling(window, min_periods=1, center=True).mean()
            X[f'ear_s{window}'] = ear_d.rolling(window, min_periods=1, center=True).std()

        X['ear_con'] = ear_d.rolling(30, min_periods=1, center=True).std() / (ear_d.rolling(30, min_periods=1, center=True).mean() + 1e-6)

        del ear_d
        gc.collect()

    if 'nose' in available_body_parts:
        nose_speed = np.sqrt(single_mouse['nose']['x'].diff()**2 + single_mouse['nose']['y'].diff()**2)
        for window in [15, 30, 60, 120]:
            X[f'nose_sp{window}'] = nose_speed.rolling(window, min_periods=5).mean()
            X[f'nose_sp_s{window}'] = nose_speed.rolling(window, min_periods=5).std()
        del nose_speed
        gc.collect()

    if 'tail_base' in available_body_parts:
        tail_speed = np.sqrt(single_mouse['tail_base']['x'].diff()**2 + single_mouse['tail_base']['y'].diff()**2)
        for window in [15, 30, 60, 120]:
            X[f'tail_sp{window}'] = tail_speed.rolling(window, min_periods=5).mean()
            X[f'tail_sp_s{window}'] = tail_speed.rolling(window, min_periods=5).std()
        del tail_speed
        gc.collect()

    X = X.astype(np.float32)
    gc.collect()
    return X

def transform_pair(mouse_pair, body_parts_tracked):
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    }, dtype=np.float32)
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        for shift in [5, 10, 20]:
            shA = mouse_pair['A']['ear_left'].shift(shift)
            shB = mouse_pair['B']['ear_left'].shift(shift)
            speeds = pd.DataFrame({
                f'sp_A{shift}': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
                f'sp_AB{shift}': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
                f'sp_B{shift}': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
            }, dtype=np.float32)
            X = pd.concat([X, speeds], axis=1)
            del shA, shB, speeds
            gc.collect()

    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

        for window in [15, 30, 60, 120]:
            X[f'rel_ori_m{window}'] = X['rel_ori'].rolling(window, min_periods=5).mean()
            X[f'rel_ori_s{window}'] = X['rel_ori'].rolling(window, min_periods=5).std()

        del dir_A, dir_B
        gc.collect()

    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)

        for lag in [5, 10, 20, 40, 80]:
            shA_n = mouse_pair['A']['nose'].shift(lag)
            shB_n = mouse_pair['B']['nose'].shift(lag)
            past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
            X[f'appr{lag}'] = cur - past
            del shA_n, shB_n, past
            gc.collect()

        del cur
        gc.collect()

    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                    (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)

        X['v_cls'] = (cd < 5.0).astype(np.float32)
        X['cls'] = ((cd >= 5.0) & (cd < 15.0)).astype(np.float32)
        X['med'] = ((cd >= 15.0) & (cd < 30.0)).astype(np.float32)
        X['far'] = (cd >= 30.0).astype(np.float32)

        for window in [30, 60, 120, 240]:
            X[f'v_cls_m{window}'] = X['v_cls'].rolling(window, min_periods=5).mean()
            X[f'cls_m{window}'] = X['cls'].rolling(window, min_periods=5).mean()
            X[f'med_m{window}'] = X['med'].rolling(window, min_periods=5).mean()

        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)

        for w in [5, 10, 15, 30, 60, 120, 240]:
            X[f'd_m{w}'] = cd_full.rolling(w, min_periods=1, center=True).mean()
            X[f'd_s{w}'] = cd_full.rolling(w, min_periods=1, center=True).std()
            X[f'd_mn{w}'] = cd_full.rolling(w, min_periods=1, center=True).min()
            X[f'd_mx{w}'] = cd_full.rolling(w, min_periods=1, center=True).max()

            d_var = cd_full.rolling(w, min_periods=1, center=True).var()
            X[f'int{w}'] = 1 / (1 + d_var)
            del d_var
            gc.collect()

        Axd = mouse_pair['A']['body_center']['x'].diff()
        Ayd = mouse_pair['A']['body_center']['y'].diff()
        Bxd = mouse_pair['B']['body_center']['x'].diff()
        Byd = mouse_pair['B']['body_center']['y'].diff()

        for w in [5, 10, 15, 30, 60, 120]:
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{w}'] = coord.rolling(w, min_periods=1, center=True).mean()
            X[f'co_s{w}'] = coord.rolling(w, min_periods=1, center=True).std()
            del coord
            gc.collect()

        cd_change = cd.diff()
        for window in [30, 60, 120]:
            X[f'cd_ch{window}'] = cd_change.rolling(window, min_periods=5).sum()
            X[f'cd_ch_s{window}'] = cd_change.rolling(window, min_periods=5).std()

        val = (Axd * Bxd + Ayd * Byd) / (np.sqrt(Axd**2 + Ayd**2) * np.sqrt(Bxd**2 + Byd**2) + 1e-6)

        for off in [-40, -20, -10, 0, 10, 20, 40]:
            X[f'va_{off}'] = val.shift(-off)

        for window in [30, 60, 120]:
            X[f'va_m{window}'] = val.rolling(window, min_periods=5).mean()
            X[f'va_s{window}'] = val.rolling(window, min_periods=5).std()

        A_speed = np.sqrt(Axd**2 + Ayd**2)
        B_speed = np.sqrt(Bxd**2 + Byd**2)
        X['speed_ratio'] = A_speed / (B_speed + 1e-6)
        X['speed_diff'] = A_speed - B_speed

        for window in [30, 60, 120]:
            X[f'A_sp{window}'] = A_speed.rolling(window, min_periods=5).mean()
            X[f'B_sp{window}'] = B_speed.rolling(window, min_periods=5).mean()
            X[f'sp_rat{window}'] = (A_speed / (B_speed + 1e-6)).rolling(window, min_periods=5).mean()

        X = add_interaction_features(X, mouse_pair, avail_A, avail_B)
        X = add_pair_synchrony_features(X, mouse_pair, avail_A, avail_B)
        X = add_relative_motion_features(X, mouse_pair, avail_A, avail_B)

        del cd, cd_full, cd_change, Axd, Ayd, Bxd, Byd, val, A_speed, B_speed
        gc.collect()

    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                    (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)

        for lag in [5, 10, 20, 40, 80]:
            X[f'nn_lg{lag}'] = nn.shift(lag)
            X[f'nn_ch{lag}'] = nn - nn.shift(lag)

            is_cl = (nn < 10.0).astype(np.float32)
            X[f'cl_ps{lag}'] = is_cl.rolling(lag, min_periods=1).mean()
            del is_cl
            gc.collect()

        for window in [30, 60, 120, 240]:
            X[f'nn_m{window}'] = nn.rolling(window, min_periods=5).mean()
            X[f'nn_s{window}'] = nn.rolling(window, min_periods=5).std()
            X[f'nn_mn{window}'] = nn.rolling(window, min_periods=5).min()

        del nn
        gc.collect()

    if 'nose' in avail_A and 'body_center' in avail_B:
        nose_to_body = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                               (mouse_pair['A']['nose']['y'] - mouse_pair['B']['body_center']['y'])**2)
        for window in [30, 60, 120]:
            X[f'nb_m{window}'] = nose_to_body.rolling(window, min_periods=5).mean()
            X[f'nb_s{window}'] = nose_to_body.rolling(window, min_periods=5).std()
        del nose_to_body
        gc.collect()

    if 'body_center' in avail_A and 'nose' in avail_B:
        body_to_nose = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['nose']['x'])**2 +
                               (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['nose']['y'])**2)
        for window in [30, 60, 120]:
            X[f'bn_m{window}'] = body_to_nose.rolling(window, min_periods=5).mean()
            X[f'bn_s{window}'] = body_to_nose.rolling(window, min_periods=5).std()
        del body_to_nose
        gc.collect()

    X = X.astype(np.float32)
    gc.collect()
    return X

In [8]:
def train_cat_model_2(X_tr, label, base_subsample):
    model = make_pipeline(
        SimpleImputer(strategy='median'),
        StratifiedSubsetClassifier(
            CatBoostClassifier(
                iterations=280,
                learning_rate=0.05,
                depth=10,
                l2_leaf_reg=3.5,
                random_strength=0.6,
                bagging_temperature=0.8,
                task_type='GPU',
                verbose=False,
                allow_writing_files=False,
                random_seed=SEED + 1
            ),
            base_subsample,
            random_state=SEED + 1
        )
    )
    
    model_list = []
    action_counter = 0

    for action in label.columns:
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(np.int8)

        if not (y_action == 0).all() and y_action.sum() >= 5:
            m_clone = clone(model)
            m_clone.fit(X_tr[action_mask], y_action)
            model_list.append((action, m_clone))
            action_counter += 1

            if action_counter % 2 == 0:
                mem_current = psutil.virtual_memory().percent
                print(f"    Trained {action_counter} actions, Memory: {mem_current}%")
                gc.collect()
                if CUPY_AVAILABLE:
                    cp.get_default_memory_pool().free_all_blocks()

            del y_action, m_clone

    return model_list

In [9]:
os.makedirs('/kaggle/working/models', exist_ok=True)

print(f"Starting CatBoost Model 2 Training")
print(f"Starting Memory: {psutil.virtual_memory().percent}%\n")

for section in range(1, len(body_parts_tracked_list)):
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing: {len(body_parts_tracked)} body parts")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]
        single_list, single_label_list, single_meta_list = [], [], []
        pair_list, pair_label_list, pair_meta_list = [], [], []

        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_list.append(data)
                single_meta_list.append(meta)
                single_label_list.append(label)
            else:
                pair_list.append(data)
                pair_meta_list.append(meta)
                pair_label_list.append(label)

        if len(single_list) > 0:
            single_mouse = pd.concat(single_list)
            single_label = pd.concat(single_label_list)
            single_meta = pd.concat(single_meta_list)
            del single_list, single_label_list, single_meta_list
            gc.collect()
            if CUPY_AVAILABLE:
                cp.get_default_memory_pool().free_all_blocks()

            X_tr = transform_single(single_mouse, body_parts_tracked)
            del single_mouse
            gc.collect()

            print(f"  Single: {X_tr.shape}")
            
            data_size = len(X_tr)
            print(f"  Data size: {data_size:,} rows")

            if data_size > 2500000:
                max_samples = 2000000
                np.random.seed(SEED)
                sample_idx = np.random.choice(data_size, min(max_samples, data_size), replace=False)
                sample_idx = np.sort(sample_idx)
    
                X_tr = X_tr.iloc[sample_idx].reset_index(drop=True)
                single_label = single_label.iloc[sample_idx].reset_index(drop=True)
                single_meta = single_meta.iloc[sample_idx].reset_index(drop=True)
    
                print(f"  Downsampled to: {len(X_tr):,} rows (ratio: {len(X_tr)/data_size:.2%})")
                del sample_idx
                gc.collect()

            base_subsample = 1200000 if data_size > 2000000 else 1500000
            model_list = train_cat_model_2(X_tr, single_label, base_subsample)
            
            with open(f'/kaggle/working/models/cat2_single_{section}.pkl', 'wb') as f:
                pickle.dump(model_list, f)
            
            del X_tr, single_label, single_meta, model_list
            gc.collect()
            if CUPY_AVAILABLE:
                cp.get_default_memory_pool().free_all_blocks()

        if len(pair_list) > 0:
            mouse_pair = pd.concat(pair_list)
            pair_label = pd.concat(pair_label_list)
            pair_meta = pd.concat(pair_meta_list)
            del pair_list, pair_label_list, pair_meta_list
            gc.collect()
            if CUPY_AVAILABLE:
                cp.get_default_memory_pool().free_all_blocks()

            X_tr = transform_pair(mouse_pair, body_parts_tracked)
            del mouse_pair
            gc.collect()

            print(f"  Pair: {X_tr.shape}")
            
            data_size = len(X_tr)
            print(f"  Data size: {data_size:,} rows")

            if data_size > 2500000:
                max_samples = 2000000
                np.random.seed(SEED)
                sample_idx = np.random.choice(data_size, min(max_samples, data_size), replace=False)
                sample_idx = np.sort(sample_idx)
    
                X_tr = X_tr.iloc[sample_idx].reset_index(drop=True)
                pair_label = pair_label.iloc[sample_idx].reset_index(drop=True)
                pair_meta = pair_meta.iloc[sample_idx].reset_index(drop=True)
    
                print(f"  Downsampled to: {len(X_tr):,} rows (ratio: {len(X_tr)/data_size:.2%})")
                del sample_idx
                gc.collect()

            base_subsample = 1200000 if data_size > 2000000 else 1500000
            model_list = train_cat_model_2(X_tr, pair_label, base_subsample)
            
            with open(f'/kaggle/working/models/cat2_pair_{section}.pkl', 'wb') as f:
                pickle.dump(model_list, f)
            
            del X_tr, pair_label, pair_meta, model_list
            gc.collect()
            if CUPY_AVAILABLE:
                cp.get_default_memory_pool().free_all_blocks()

        mem_current = psutil.virtual_memory().percent
        print(f"  Section {section} complete. Memory: {mem_current}%")

    except Exception as e:
        print(f'***Exception*** {str(e)[:100]}')
        import traceback
        traceback.print_exc()

    gc.collect()
    if CUPY_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
    print()

print(f"\nCatBoost Model 2 Training Complete")
print(f"Final Memory: {psutil.virtual_memory().percent}%")

Starting CatBoost Model 2 Training
Starting Memory: 4.4%

1. Processing: 18 body parts
  Single: (544859, 295)
  Data size: 544,859 rows
  Pair: (1524906, 288)
  Data size: 1,524,906 rows
    Trained 2 actions, Memory: 19.5%
    Trained 4 actions, Memory: 19.6%
    Trained 6 actions, Memory: 19.5%
  Section 1 complete. Memory: 7.9%

2. Processing: 14 body parts
  Single: (478728, 304)
  Data size: 478,728 rows
  Pair: (613716, 307)
  Data size: 613,716 rows
    Trained 2 actions, Memory: 11.6%
  Section 2 complete. Memory: 7.1%

3. Processing: 10 body parts
  Single: (1941885, 295)
  Data size: 1,941,885 rows
  Pair: (5607030, 288)
  Data size: 5,607,030 rows
  Downsampled to: 2,000,000 rows (ratio: 35.67%)
    Trained 2 actions, Memory: 22.8%
    Trained 4 actions, Memory: 22.8%
    Trained 6 actions, Memory: 22.9%
  Section 3 complete. Memory: 16.0%

4. Processing: 8 body parts
  Pair: (2210177, 271)
  Data size: 2,210,177 rows
    Trained 2 actions, Memory: 24.6%
    Trained 4 actio